In [1]:
import gurobipy as gp
from gurobipy import GRB


In [2]:
# Resource and job sets
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']


In [3]:
# Matching score data
combinations, scores = gp.multidict({
    ('Carlos', 'Tester'): 53,
    ('Carlos', 'JavaDeveloper'): 27,
    ('Carlos', 'Architect'): 13,
    ('Joe', 'Tester'): 80,
    ('Joe', 'JavaDeveloper'): 47,
    ('Joe', 'Architect'): 67,
    ('Monika', 'Tester'): 53,
    ('Monika', 'JavaDeveloper'): 73,
    ('Monika', 'Architect'): 47
})


In [4]:
# Declare and initialize model
m = gp.Model('RAP')


Restricted license - for non-production use only - expires 2023-10-25


In [5]:
combinations


<gurobi.tuplelist (9 tuples, 2 values each):
 ( Carlos , Tester        )
 ( Carlos , JavaDeveloper )
 ( Carlos , Architect     )
 ( Joe    , Tester        )
 ( Joe    , JavaDeveloper )
 ( Joe    , Architect     )
 ( Monika , Tester        )
 ( Monika , JavaDeveloper )
 ( Monika , Architect     )
>

In [6]:
# Create decision variables for the RAP model
x = m.addVars(combinations, name="assign")
x

{('Carlos', 'Tester'): <gurobi.Var *Awaiting Model Update*>,
 ('Carlos', 'JavaDeveloper'): <gurobi.Var *Awaiting Model Update*>,
 ('Carlos', 'Architect'): <gurobi.Var *Awaiting Model Update*>,
 ('Joe', 'Tester'): <gurobi.Var *Awaiting Model Update*>,
 ('Joe', 'JavaDeveloper'): <gurobi.Var *Awaiting Model Update*>,
 ('Joe', 'Architect'): <gurobi.Var *Awaiting Model Update*>,
 ('Monika', 'Tester'): <gurobi.Var *Awaiting Model Update*>,
 ('Monika', 'JavaDeveloper'): <gurobi.Var *Awaiting Model Update*>,
 ('Monika', 'Architect'): <gurobi.Var *Awaiting Model Update*>}

In [8]:
# Create job constraints
jobs = m.addConstrs((x.sum('*',j) == 1 for j in J), name='job')
jobs


{'Tester': <gurobi.Constr *Awaiting Model Update*>,
 'JavaDeveloper': <gurobi.Constr *Awaiting Model Update*>,
 'Architect': <gurobi.Constr *Awaiting Model Update*>}

In [9]:
# Create resource constraints
resources = m.addConstrs((x.sum(r,'*') <= 1 for r in R), name='resource')
resources


{'Carlos': <gurobi.Constr *Awaiting Model Update*>,
 'Joe': <gurobi.Constr *Awaiting Model Update*>,
 'Monika': <gurobi.Constr *Awaiting Model Update*>}

In [10]:
scores


{('Carlos', 'Tester'): 53,
 ('Carlos', 'JavaDeveloper'): 27,
 ('Carlos', 'Architect'): 13,
 ('Joe', 'Tester'): 80,
 ('Joe', 'JavaDeveloper'): 47,
 ('Joe', 'Architect'): 67,
 ('Monika', 'Tester'): 53,
 ('Monika', 'JavaDeveloper'): 73,
 ('Monika', 'Architect'): 47}

In [11]:
x.prod(scores)


<gurobi.LinExpr: 53.0 <gurobi.Var *Awaiting Model Update*> + 27.0 <gurobi.Var *Awaiting Model Update*> + 13.0 <gurobi.Var *Awaiting Model Update*> + 80.0 <gurobi.Var *Awaiting Model Update*> + 47.0 <gurobi.Var *Awaiting Model Update*> + 67.0 <gurobi.Var *Awaiting Model Update*> + 53.0 <gurobi.Var *Awaiting Model Update*> + 73.0 <gurobi.Var *Awaiting Model Update*> + 47.0 <gurobi.Var *Awaiting Model Update*>>

In [12]:
# Objective: maximize total matching score of all assignments
m.setObjective(x.prod(scores), GRB.MAXIMIZE)


In [13]:
# Save model for inspection
m.write('RAP.lp')

m.display()


Maximize
<gurobi.LinExpr: 53.0 assign[Carlos,Tester] + 27.0 assign[Carlos,JavaDeveloper]
+ 13.0 assign[Carlos,Architect] + 80.0 assign[Joe,Tester]
+ 47.0 assign[Joe,JavaDeveloper] + 67.0 assign[Joe,Architect]
+ 53.0 assign[Monika,Tester] + 73.0 assign[Monika,JavaDeveloper]
+ 47.0 assign[Monika,Architect]>
Subject To
job[Tester]: <gurobi.LinExpr: assign[Carlos,Tester] + assign[Joe,Tester] +
 assign[Monika,Tester]> = 1
job[JavaDeveloper]: <gurobi.LinExpr: assign[Carlos,JavaDeveloper] +
 assign[Joe,JavaDeveloper] + assign[Monika,JavaDeveloper]> = 1
job[Architect]: <gurobi.LinExpr: assign[Carlos,Architect] + assign[Joe,Architect] +
 assign[Monika,Architect]> = 1
job[Tester]: <gurobi.LinExpr: assign[Carlos,Tester] + assign[Joe,Tester] +
 assign[Monika,Tester]> = 1
job[JavaDeveloper]: <gurobi.LinExpr: assign[Carlos,JavaDeveloper] +
 assign[Joe,JavaDeveloper] + assign[Monika,JavaDeveloper]> = 1
job[Architect]: <gurobi.LinExpr: assign[Carlos,Architect] + assign[Joe,Architect] +
 assign[Monika,

In [14]:
# Run optimization engine
m.optimize()


Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 9 rows, 9 columns and 27 nonzeros
Model fingerprint: 0xa4a1434c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 3 rows and 0 columns
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2000000e+02   1.000000e+00   0.000000e+00      0s
       2    1.9300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.930000000e+02


In [15]:
# Display optimal values of decision variables
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# Display optimal total matching score
print('Total matching score: ', m.objVal)


assign[Carlos,Tester] 1.0
assign[Joe,Architect] 1.0
assign[Monika,JavaDeveloper] 1.0
Total matching score:  193.0
